In [1]:



# --- BLOCK 1 (ROBUST): Load NGFS short-term, normalize country names,
#                       keep only direct_impacts, and select the right DAPS per country.

import pandas as pd
import numpy as np
from pathlib import Path

ROOT    = Path.home() / "Documents" / "Summer_intern"
IN_XLSX = ROOT / "NGFS_shortterm_FULL_timeseries.xlsx"
DAPS_CSV = ROOT / "chosen_scenarios_by_country.csv"

YEARS = list(range(2023, 2031))
FAMILY_SHEETS = [
    "capital_destruction","production_lost","productivity_loss","labour_productivity_loss"
]

# --- Load the four sheets (minimal columns) ---
frames = []
for sheet in FAMILY_SHEETS:
    df = pd.read_excel(
        IN_XLSX, sheet_name=sheet,
        usecols=["family","hazard","sector","model","scenario","region","unit","year","value"]
    )
    df.columns = [c.strip().lower() for c in df.columns]
    if "family" not in df.columns:
        df["family"] = sheet
    frames.append(df)

full0 = pd.concat(frames, ignore_index=True)

# Coerce types and basic filters
full0["year"]  = pd.to_numeric(full0["year"], errors="coerce").astype("Int64")
full0["value"] = pd.to_numeric(full0["value"], errors="coerce")
full0 = full0[full0["year"].isin(YEARS)]
full0 = full0[full0["model"].fillna("").str.lower().eq("direct_impacts")].copy()

# Extract country name from "Country - ISO3" (or just "USA" etc.)
def country_from_region_str(s: str) -> str:
    return str(s).split(" - ")[0].strip()

full0["country"] = full0["region"].apply(country_from_region_str)

# --- Normalize country labels (so keys match CSVs and plots) ---
alias = {
    "United States": "USA",
    "U.S.A.": "USA",
    "US": "USA",
    "United States of America": "USA",
    "Korea, Republic of": "South Korea",
    "Republic of Korea": "South Korea",
    "Russian Federation": "Russia",
    "Viet Nam": "Vietnam",
}
full0["country"] = full0["country"].astype(str).str.strip().replace(alias)

# --- Load chosen DAPS per country ---
daps = pd.read_csv(DAPS_CSV)
daps["country"] = daps["country"].astype(str).str.strip().replace(alias)
daps["chosen_daps"] = daps["chosen_daps"].astype(str).str.strip()

# Sanity: ensure all countries in data appear in the mapping
missing_map = sorted(set(full0["country"]) - set(daps["country"]))
if missing_map:
    raise RuntimeError(f"Countries missing in chosen_scenarios_by_country.csv: {missing_map}")

# Tolerant scenario picker: exact -> prefix -> best DAPS by total value
def pick_scenario_for_country(df_c, chosen):
    # exact
    exact = df_c.loc[df_c["scenario"].eq(chosen)]
    if not exact.empty:
        return exact
    # prefix match
    pref = df_c.loc[df_c["scenario"].str.startswith(chosen)]
    if not pref.empty:
        return pref
    # fallback: among DAPS_* only, keep the scenario with largest total value
    daps_only = df_c.loc[df_c["scenario"].str.startswith("DAPS_")].copy()
    if daps_only.empty:
        return df_c.iloc[0:0]
    scores = (daps_only.groupby("scenario", as_index=False)["value"].sum()
                        .sort_values("value", ascending=False))
    best = scores["scenario"].iloc[0]
    return daps_only.loc[daps_only["scenario"].eq(best)]

full0["scenario"] = full0["scenario"].astype(str).str.strip()
kept_parts = []
for c, dfc in full0.groupby("country", sort=False):
    chosen = daps.loc[daps["country"].eq(c), "chosen_daps"].iloc[0]
    kept = pick_scenario_for_country(dfc, chosen)
    if kept.empty:
        print(f"[WARN] No matching DAPS kept for {c} (chosen={chosen}); country dropped")
    else:
        kept_parts.append(kept)

df_block1 = pd.concat(kept_parts, ignore_index=True)

print("BLOCK 1 OK — robust DAPS selection")
print("Rows after filter:", len(df_block1))
print("Countries retained:", df_block1['country'].nunique())
print("Sample countries:", sorted(df_block1['country'].unique().tolist())[:15])
print("Hazards present:", sorted(df_block1['hazard'].dropna().unique().tolist()))


BLOCK 1 OK — robust DAPS selection
Rows after filter: 493800
Countries retained: 46
Sample countries: ['Argentina', 'Austria', 'Belgium', 'Brazil', 'Bulgaria', 'Canada', 'China', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany']
Hazards present: ['coastal_flood', 'drought', 'floods', 'heatwave', 'river_flood', 'storms', 'tropical_cyclone', 'wildfire', 'winter_storm']


In [2]:
# --- BLOCK 2: lock sector universe to your 10 canonical sectors (with aliases) and filter ---

import re
import pandas as pd
import numpy as np

df = df_block1.copy()

# Canonical sector order you requested
TARGET_SECTOR_CANON = [
    "Power Supply",
    "Ferrous metals",
    "Non-metallic minerals",
    "Construction",
    "Land transport",
    "Water transport",
    "Warehousing",
    "Agriculture",
    "Market Services",
    "Non Market Services",
]

# Alias lists to match dataset labels (be generous but precise)
SECTOR_ALIASES = {
    "Power Supply": [
        "Power Supply", "Power supply", "Electricity", "Electric power",
        "Electricity and gas", "Electricity, gas, steam", "Electricity and heat",
        "Power generation", "Electric", "Power"
    ],
    "Ferrous metals": [
        "Ferrous metals", "Iron and steel", "Basic iron and steel",
        "Manufacture of basic iron and steel", "Basic metals (ferrous)"
    ],
    "Non-metallic minerals": [
        "Non-metallic minerals", "Other non-metallic mineral products",
        "Cement, lime and plaster", "Glass and glass products",
        "Non metallic minerals"
    ],
    "Construction": ["Construction"],
    "Land transport": ["Land transport"],
    "Water transport": ["Water transport", "Inland water transport", "Water transportation"],
    "Warehousing": ["Warehousing", "Warehousing and support activities for transportation", "Warehousing and support"],
    "Agriculture": ["Agriculture"],
    "Market Services": ["Market Services", "Market services"],
    "Non Market Services": ["Non Market Services", "Non-market services", "Non Market services"],
}

all_sectors = set(df["sector"].dropna().unique())
target_map = {}     # canonical -> dataset label actually used
missing_targets = []

def match_alias(alias_list, universe):
    # exact match (case-sensitive then insensitive), else contains-based
    for alias in alias_list:
        if alias in universe:
            return alias
        # case-insensitive exact
        for s in universe:
            if s.lower() == alias.lower():
                return s
    # contains-based fallback (lowercased)
    lowered = {s.lower(): s for s in universe}
    for alias in alias_list:
        a = alias.lower()
        for sl, orig in lowered.items():
            if a in sl:
                return orig
    return None

for canon, aliases in SECTOR_ALIASES.items():
    found = match_alias(aliases, all_sectors)
    if found is None:
        missing_targets.append(canon)
    else:
        target_map[canon] = found

print("Sector alias resolution (canonical -> dataset label):")
for k in TARGET_SECTOR_CANON:
    print(f"  {k:>22}  ->  {target_map.get(k, 'NOT FOUND')}")

if missing_targets:
    print("\n[WARN] Missing sectors (not found in data):", missing_targets)
    # If strictness is desired, uncomment the next line:
    # raise RuntimeError(f"Missing target sectors: {missing_targets}")

kept_sector_labels = [target_map[s] for s in TARGET_SECTOR_CANON if s in target_map]
df = df[df["sector"].isin(kept_sector_labels)].copy()

# For later steps, keep a reverse map dataset_label -> canonical name
inv_map = {v:k for k,v in target_map.items()}
df["sector_canon"] = df["sector"].map(inv_map)

print("\nBLOCK 2 OK — sectors filtered")
print("Rows after sector filter:", len(df))
print("Kept sectors (dataset labels, in canonical order):", kept_sector_labels)

# Expose for next blocks
df_block2 = df.copy()
target_map_block2 = target_map.copy()


Sector alias resolution (canonical -> dataset label):
            Power Supply  ->  Power Supply
          Ferrous metals  ->  Ferrous metals
   Non-metallic minerals  ->  Non-metallic minerals
            Construction  ->  Construction
          Land transport  ->  Land transport
         Water transport  ->  Water transport
             Warehousing  ->  Warehousing
             Agriculture  ->  Agriculture
         Market Services  ->  Market Services
     Non Market Services  ->  Non Market Services

BLOCK 2 OK — sectors filtered
Rows after sector filter: 98760
Kept sectors (dataset labels, in canonical order): ['Power Supply', 'Ferrous metals', 'Non-metallic minerals', 'Construction', 'Land transport', 'Water transport', 'Warehousing', 'Agriculture', 'Market Services', 'Non Market Services']


In [3]:
# --- BLOCK 3: total_loss per (country, sector_canon, hazard, year)
#              + storylines HDW/SF as PEAK (max across years) ---

import pandas as pd
import numpy as np

df = df_block2.copy()

# Sum of 4 families (equal weights) per (country, sector_canon, hazard, year)
agg = (df.groupby(["country","sector_canon","hazard","year"], dropna=False)
         .agg(total_loss=("value","sum"))
         .reset_index())

# Per-year columns for hazards (include tropical_cyclone and winter_storm)
need_hazards = [
    "floods","river_flood","coastal_flood",
    "storms","tropical_cyclone","winter_storm",
    "heatwave","drought","wildfire"
]

base = agg[["country","sector_canon","year"]].drop_duplicates()
for h in need_hazards:
    if h in agg["hazard"].unique():
        sub = (agg[agg["hazard"] == h]
               [["country","sector_canon","year","total_loss"]]
               .rename(columns={"total_loss": f"tot_{h}"}))
        base = base.merge(sub, on=["country","sector_canon","year"], how="left")
    else:
        base[f"tot_{h}"] = np.nan

# Combined flood per year = max of flood components present
flood_cols = [c for c in ["tot_floods","tot_river_flood","tot_coastal_flood"] if c in base.columns]
base["tot_flood"] = (np.nanmax(np.stack([base[c].to_numpy() for c in flood_cols], axis=1), axis=1)
                     if flood_cols else np.nan)

# Combined storms per year = max(storms, tropical_cyclone, winter_storm) if present
storms_cols = [c for c in ["tot_storms","tot_tropical_cyclone","tot_winter_storm"] if c in base.columns]
base["tot_storms_combined"] = (np.nanmax(np.stack([base[c].to_numpy() for c in storms_cols], axis=1), axis=1)
                               if storms_cols else np.nan)

# PEAK (max over years) per country–sector for each component
peaks = (base.groupby(["country","sector_canon"], dropna=False)
             .agg(
                 peak_storms   = ("tot_storms_combined", "max"),
                 peak_flood    = ("tot_flood",           "max"),
                 peak_heat     = ("tot_heatwave",        "max"),
                 peak_drought  = ("tot_drought",         "max"),
                 peak_wildfire = ("tot_wildfire",        "max"),
             )
             .reset_index())

# Storylines (sum of component peaks; treat missing as 0)
for col in ["peak_storms","peak_flood","peak_heat","peak_drought","peak_wildfire"]:
    if col not in peaks.columns:
        peaks[col] = np.nan

peaks["HDW"] = peaks[["peak_heat","peak_drought","peak_wildfire"]].fillna(0.0).sum(axis=1)
peaks["SF"]  = peaks[["peak_storms","peak_flood"]].fillna(0.0).sum(axis=1)

print("BLOCK 3 OK")
print(peaks.head(10))

# Expose for next blocks
peaks_block3 = peaks.copy()


BLOCK 3 OK
     country           sector_canon  peak_storms  peak_flood  peak_heat  \
0  Argentina            Agriculture     0.000000    0.000000   2.049540   
1  Argentina           Construction     0.000000    0.000000   4.099081   
2  Argentina         Ferrous metals     0.016398    3.519976   0.000000   
3  Argentina         Land transport     0.018220    3.911084   0.000000   
4  Argentina        Market Services     0.000000    0.000000   0.409908   
5  Argentina    Non Market Services     0.000000    0.000000   0.000000   
6  Argentina  Non-metallic minerals     0.016398    3.519976   0.000000   
7  Argentina           Power Supply     0.018220    3.911084   0.000000   
8  Argentina            Warehousing     0.018220    3.911084   0.000000   
9  Argentina        Water transport     0.018220    3.911084   0.000000   

   peak_drought  peak_wildfire        HDW        SF  
0      6.938826       4.754487  13.742853  0.000000  
1      0.000000       4.516763   8.615844  0.000000  
2

In [4]:
# --- BLOCK 4 (HARDENED): aggregate countries -> macro regions with strong normalization ---

import pandas as pd
import unicodedata
from pathlib import Path

ROOT = Path.home() / "Documents" / "Summer_intern"
MACRO_CSV = ROOT / "country_to_macro_region.csv"

peaks = peaks_block3.copy()

def norm(s: str) -> str:
    # normalize Unicode, drop NBSP and zero-widths, collapse spaces, strip
    s = unicodedata.normalize("NFKC", str(s))
    s = s.replace("\u00A0"," ").replace("\u200B","").replace("\u200C","").replace("\u200D","")
    return " ".join(s.split()).strip()

# 1) Normalize countries in peaks (same aliases as Block 1, then strong norm)
alias = {
    "United States": "USA",
    "U.S.A.": "USA",
    "US": "USA",
    "United States of America": "USA",
    "Korea, Republic of": "South Korea",
    "Republic of Korea": "South Korea",
    "Russian Federation": "Russia",
    "Viet Nam": "Vietnam",
}
peaks["country"] = peaks["country"].astype(str).map(norm).replace(alias).map(norm)

# 2) Load mapping and normalize strongly
macro_map = pd.read_csv(MACRO_CSV, dtype=str)
if "country" not in macro_map.columns or "macro_region" not in macro_map.columns:
    raise RuntimeError("country_to_macro_region.csv must have columns: country, macro_region")

macro_map["country"] = macro_map["country"].astype(str).map(norm).replace(alias).map(norm)
macro_map["macro_region"] = macro_map["macro_region"].astype(str).map(norm)

# 3) Auto-add essential rows if missing (protects against hidden chars)
need_explicit = {
    "USA": "USA",
    "Mexico": "Mexico",
    "Australia": "Australia_NZ",
    "New Zealand": "Australia_NZ",
    "Switzerland": "Europe_Non_EU",
    "Norway": "Europe_Non_EU",
    "Iceland": "Europe_Non_EU",
    "Turkey": "Europe_Non_EU",
}
missing_essentials = [k for k in need_explicit if not (macro_map["country"] == k).any()]
if missing_essentials:
    extra = pd.DataFrame({"country": missing_essentials,
                          "macro_region": [need_explicit[k] for k in missing_essentials]})
    macro_map = pd.concat([macro_map, extra], ignore_index=True)

# 4) Add mapping rows for NGFS aggregate labels (if any appear in your file)
ngfs_aggregates = {
    "Oceania": "Australia_NZ",
    "Rest of Europe": "Europe_Non_EU",
    "Rest of World": "Other",
    "Rest of Energy Producing Countries": "Other",
}
for aggr_country, macro in ngfs_aggregates.items():
    if not (macro_map["country"] == aggr_country).any():
        macro_map.loc[len(macro_map)] = {"country": aggr_country, "macro_region": macro}

# 5) Drop duplicate countries in the mapping (keep the first occurrence)
macro_map = macro_map.drop_duplicates(subset=["country"], keep="first").reset_index(drop=True)

# 6) Final sanity: make sure every country in peaks is mappable
present = set(macro_map["country"])
needed  = set(peaks["country"])
still_missing = sorted(needed - present)
if still_missing:
    # As a last resort, auto-assign to "Other" to avoid breaking the pipeline
    extra = pd.DataFrame({"country": still_missing, "macro_region": ["Other"] * len(still_missing)})
    macro_map = pd.concat([macro_map, extra], ignore_index=True)

# 7) Merge and aggregate
peaks = peaks.merge(macro_map, on="country", how="left", validate="many_to_one")
if peaks["macro_region"].isna().any():
    bad = sorted(peaks.loc[peaks["macro_region"].isna(), "country"].unique())
    raise RuntimeError(f"Unmapped after merge even after normalization: {bad}")

macro = (
    peaks.groupby(["macro_region","sector_canon"], dropna=False)
         .agg(HDW=("HDW","sum"), SF=("SF","sum"))
         .reset_index()
)

# --- Replace 'Other' with MEAN across its member countries (consistency with EU means) ---
# Use `peaks`, which already includes `macro_region` from step 7.
other_mean = (peaks.loc[peaks["macro_region"] == "Other"]
              .groupby(["sector_canon"], dropna=False)
              .agg(HDW=("HDW","mean"), SF=("SF","mean"))
              .reset_index())

if not other_mean.empty:
    other_mean.insert(0, "macro_region", "Other")
    macro = pd.concat([macro[macro["macro_region"] != "Other"], other_mean], ignore_index=True)


print("BLOCK 4 OK (hardened)")
print("Macro regions present:", sorted(macro["macro_region"].unique().tolist()))

macro_block4 = macro.copy()


BLOCK 4 OK (hardened)
Macro regions present: ['Argentina', 'Australia_NZ', 'Brazil', 'Canada', 'China', 'EU-27', 'Europe_Non_EU', 'India', 'Indonesia', 'Japan', 'Mexico', 'Other', 'Russia', 'South Africa', 'South Korea', 'USA']


In [5]:
# --- BLOCK 4A: add EU-15 and EU-27 (means across member countries) ---

import pandas as pd

peaks_c = peaks_block3.copy()  # raw country-level peaks from Block 3

# Robust country labels (match what appears in df_block1/peaks)
# EU-15 = pre-2004 members (historical definition, includes UK)
EU15 = [
    "Austria","Belgium","Denmark","Finland","France","Germany","Greece",
    "Ireland","Italy","Luxembourg","Netherlands","Portugal","Spain","Sweden",
    "United Kingdom"
]

# EU-27 current members (2007+; exclude UK, include the 27 today)
EU27 = [
    "Austria","Belgium","Bulgaria","Croatia","Cyprus","Czechia","Denmark",
    "Estonia","Finland","France","Germany","Greece","Hungary","Ireland",
    "Italy","Latvia","Lithuania","Luxembourg","Malta","Netherlands","Poland",
    "Portugal","Romania","Slovakia","Slovenia","Spain","Sweden"
]

def add_macro_mean(peaks_df, members, macro_name):
    present = set(peaks_df["country"])
    used = [c for c in members if c in present]
    if not used:
        return None
    sub = peaks_df[peaks_df["country"].isin(used)]
    # mean across countries for each sector
    agg_mean = (sub.groupby(["sector_canon"], dropna=False)
                  .agg(HDW=("HDW","mean"), SF=("SF","mean"))
                  .reset_index())
    agg_mean.insert(0, "macro_region", macro_name)
    return agg_mean

eu15_rows = add_macro_mean(peaks_c, EU15, "EU-15")
eu27_rows = add_macro_mean(peaks_c, EU27, "EU-27")

# Start from the macro produced in Block 4 (which used sum by mapping)
macro_aug = macro.copy()

# If EU-27 already exists from the mapping (sum), drop it and replace by mean
if "EU-27" in macro_aug["macro_region"].unique():
    macro_aug = macro_aug[macro_aug["macro_region"] != "EU-27"]

# Append the new aggregates if available
to_add = []
if eu15_rows is not None: to_add.append(eu15_rows)
if eu27_rows is not None: to_add.append(eu27_rows)
if to_add:
    macro_aug = pd.concat([macro_aug] + to_add, ignore_index=True)

# Expose for Block 5
macro_block4 = macro_aug.copy()
print("BLOCK 4A OK — EU-15 and mean EU-27 added where possible.")
print("Macro regions now:", sorted(macro_block4["macro_region"].unique()))

# DAPS
macro_daps = macro_block4.copy()   # this is the one with EU-15/EU-27 means



BLOCK 4A OK — EU-15 and mean EU-27 added where possible.
Macro regions now: ['Argentina', 'Australia_NZ', 'Brazil', 'Canada', 'China', 'EU-15', 'EU-27', 'Europe_Non_EU', 'India', 'Indonesia', 'Japan', 'Mexico', 'Other', 'Russia', 'South Africa', 'South Korea', 'USA']


In [6]:
# --- BLOCK 5: plot HDW and SF heatmaps (Regions x Sectors), export PNG + PDF ---



# ------- shared CSV helpers (place once near top of file) -------
from pathlib import Path
import pandas as pd

def _ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)
    return p

def _to_long_from_matrix(mat_df: pd.DataFrame, index_name="Region", col_name="Sector", value_name="Value"):
    idx = mat_df.index.name or index_name
    return (mat_df.reset_index()
            .rename(columns={idx: index_name})
            .melt(id_vars=index_name, var_name=col_name, value_name=value_name))

def write_heatmap_tables(base_outdir: Path, tag: str, mat_df: pd.DataFrame):
    """Write wide + long CSVs next to your PNG/PDF outputs."""
    data_dir = _ensure_dir(base_outdir / "data")
    (data_dir / f"{tag}_heatmap_data.csv").write_text(mat_df.to_csv())
    _to_long_from_matrix(mat_df).to_csv(data_dir / f"{tag}_heatmap_data_long.csv", index=False)



import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import FuncFormatter
from pathlib import Path

ROOT    = Path.home() / "Documents" / "Summer_intern"
OUTROOT = ROOT / "outputs" / "physical_shortterm_DAPS"
OUTPNG  = OUTROOT / "png"
OUTPDF  = OUTROOT / "pdf"
OUTPNG.mkdir(parents=True, exist_ok=True)
OUTPDF.mkdir(parents=True, exist_ok=True)

macro = macro_block4.copy()

# Fixed row order = same as transition + Indonesia (and optionally "Other" last if present)
region_order = [
    "USA","EU-15","EU-27","Europe_Non_EU","Japan","China","India","South Korea",
    "Canada","Australia_NZ","Mexico","Russia","Brazil",
    "South Africa","Argentina","Indonesia","Other"
]
region_order = [r for r in region_order if r in set(macro_block4["macro_region"])]


# Fixed column order = your 10 canonical sectors
sector_order = [
    "Power Supply","Ferrous metals","Non-metallic minerals","Construction",
    "Land transport","Water transport","Warehousing","Agriculture",
    "Market Services","Non Market Services",
]
sector_order = [s for s in sector_order if s in set(macro["sector_canon"])]

def matrix_from(df, value_col):
    p = df.pivot_table(index="macro_region", columns="sector_canon", values=value_col, aggfunc="sum")
    p = p.reindex(index=region_order, columns=sector_order)
    return p

M_HDW = matrix_from(macro, "HDW")
M_SF  = matrix_from(macro, "SF")

# Save CSVs (wide + long) so Streamlit can render + download
_ensure_dir(OUTROOT / "data")
write_heatmap_tables(OUTROOT, "HDW_regionsxsectors_peak", M_HDW)
write_heatmap_tables(OUTROOT, "SF_regionsxsectors_peak",  M_SF)


def p95(M):
    z = M[np.isfinite(M)]
    return float(np.percentile(z,95)) if z.size else 1.0

def draw(ax, M, rows, cols, title, unit="percent"):
    if M.size == 0:
        ax.axis("off"); ax.set_title(f"{title}\n(no data)"); return
    vmin, vmax = 0.0, p95(M)

    # heatmap
    im = ax.imshow(M, aspect="auto", cmap="RdYlGn_r", vmin=vmin, vmax=vmax)

    # ticks and labels
    ax.set_xticks(range(len(cols))); ax.set_xticklabels(cols, rotation=45, ha="right")
    ax.set_yticks(range(len(rows))); ax.set_yticklabels(rows)

    # cadrillage: put minor ticks at cell boundaries and draw a grid on them
    nrows, ncols = M.shape
    ax.set_xticks(np.arange(-0.5, ncols, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, nrows, 1), minor=True)
    ax.grid(which="minor", color="black", linewidth=0.4, linestyle="-")
    ax.tick_params(which="minor", bottom=False, left=False)

    # colorbar with simple formatting
    c = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04, format=FuncFormatter(lambda v, _pos: f"{v:.1f}"))
    c.ax.set_ylabel(f"Exposure ({unit})", rotation=270, va="bottom")

    ax.set_title(title, pad=10)

# HDW
fig1 = plt.figure(figsize=(12, max(5.0, 0.36*len(M_HDW.index)+2.2)))
ax1 = fig1.add_subplot(111)
draw(ax1, M_HDW.to_numpy(float), list(M_HDW.index), list(M_HDW.columns),
     "Short-term physical risk (PEAK) DAPS — HDW (heatwave + drought + wildfire)")
fig1.tight_layout()
png_hdw = OUTPNG / "total_loss__HDW__regionsxsectors__peak.png"
fig1.savefig(png_hdw, dpi=240); plt.close(fig1)

# SF
fig2 = plt.figure(figsize=(12, max(5.0, 0.36*len(M_SF.index)+2.2)))
ax2 = fig2.add_subplot(111)
draw(ax2, M_SF.to_numpy(float), list(M_SF.index), list(M_SF.columns),
     "Short-term physical risk (PEAK) DAPS — SF (storms + flood)")
fig2.tight_layout()
png_sf = OUTPNG / "total_loss__SF__regionsxsectors__peak.png"
fig2.savefig(png_sf, dpi=240); plt.close(fig2)

# Bundle into one PDF
pdf = OUTPDF / "total_loss__HDW_and_SF__regionsxsectors__peak.pdf"
with PdfPages(pdf) as pp:
    for png in [png_hdw, png_sf]:
        img = plt.imread(png)
        fig = plt.figure(figsize=(12,9)); plt.imshow(img); plt.axis('off')
        pp.savefig(fig); plt.close(fig)

print("BLOCK 5 OK")
print("PNG saved:", png_hdw)
print("PNG saved:", png_sf)
print("PDF saved:", pdf)


BLOCK 5 OK
PNG saved: /Users/noenotter/Documents/Summer_intern/outputs/physical_shortterm_DAPS/png/total_loss__HDW__regionsxsectors__peak.png
PNG saved: /Users/noenotter/Documents/Summer_intern/outputs/physical_shortterm_DAPS/png/total_loss__SF__regionsxsectors__peak.png
PDF saved: /Users/noenotter/Documents/Summer_intern/outputs/physical_shortterm_DAPS/pdf/total_loss__HDW_and_SF__regionsxsectors__peak.pdf


In [7]:
# ============================
# PHYSICAL RISK — DIRE (DiRe) PARALLEL PIPELINE
# ============================
import pandas as pd
import numpy as np
import unicodedata
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import FuncFormatter

# ---- Paths (keep aligned with your DAPS setup) ----
ROOT      = Path.home() / "Documents" / "Summer_intern"
IN_XLSX   = ROOT / "NGFS_shortterm_FULL_timeseries.xlsx"
MACRO_CSV = ROOT / "country_to_macro_region.csv"
OUTROOT   = ROOT / "outputs" / "physical_shortterm_DIRE"
OUTPNG    = OUTROOT / "png"
OUTPDF    = OUTROOT / "pdf"
OUTPNG.mkdir(parents=True, exist_ok=True)
OUTPDF.mkdir(parents=True, exist_ok=True)

YEARS = list(range(2023, 2031))
FAMILY_SHEETS = ["capital_destruction","production_lost","productivity_loss","labour_productivity_loss"]

def _norm(s: str) -> str:
    s = unicodedata.normalize("NFKC", str(s))
    s = s.replace("\u00A0"," ").replace("\u200B","").replace("\u200C","").replace("\u200D","")
    return " ".join(s.split()).strip()

alias = {
    "United States":"USA","U.S.A.":"USA","US":"USA","United States of America":"USA",
    "Korea, Republic of":"South Korea","Republic of Korea":"South Korea",
    "Russian Federation":"Russia","Viet Nam":"Vietnam",
}

def _build_full0():
    frames = []
    for sheet in FAMILY_SHEETS:
        df = pd.read_excel(
            IN_XLSX, sheet_name=sheet,
            usecols=["family","hazard","sector","model","scenario","region","unit","year","value"]
        )
        df.columns = [c.strip().lower() for c in df.columns]
        if "family" not in df.columns:
            df["family"] = sheet
        frames.append(df)
    f0 = pd.concat(frames, ignore_index=True)
    f0["year"]  = pd.to_numeric(f0["year"], errors="coerce").astype("Int64")
    f0["value"] = pd.to_numeric(f0["value"], errors="coerce")
    f0 = f0[f0["year"].isin(YEARS)]
    f0 = f0[f0["model"].fillna("").str.lower().eq("direct_impacts")].copy()
    # country from "Name - ISO3"
    f0["country"] = f0["region"].astype(str).str.split(" - ").str[0].str.strip()
    f0["country"] = f0["country"].replace(alias).map(_norm)
    return f0

# Reuse DAPS Block-1 output if present; else rebuild
if "full0" not in globals():
    full0 = _build_full0()

# -----------------------
# BLOCK 1 — Filter DiRe (case-insensitive)
# -----------------------
if "scenario" not in full0.columns:
    raise RuntimeError("Expected 'scenario' in full0. Did the load block run?")
scenario_norm = full0["scenario"].astype(str).str.strip().str.lower()
df = full0.loc[scenario_norm.eq("dire")].copy()
if df.empty:
    # very tolerant fallback
    df = full0.loc[full0["scenario"].astype(str).str.lower().str.contains("dire")].copy()
if df.empty:
    raise RuntimeError("No rows for DiRe/DIRE scenario found in the workbook.")

# -----------------------
# BLOCK 2 — Map to the 10 canonical sectors (same as your DAPS)
# -----------------------
TARGET_SECTOR_CANON = [
    "Power Supply","Ferrous metals","Non-metallic minerals","Construction",
    "Land transport","Water transport","Warehousing","Agriculture",
    "Market Services","Non Market Services",
]
SECTOR_ALIASES = {
    "Power Supply": ["Power Supply","Power supply","Electricity","Electric power","Electric"],
    "Ferrous metals": ["Ferrous metals","Iron and steel","Basic iron and steel"],
    "Non-metallic minerals": ["Non-metallic minerals","Other non-metallic","Cement","Glass"],
    "Construction": ["Construction"],
    "Land transport": ["Land transport"],
    "Water transport": ["Water transport","Inland water transport","Water transportation"],
    "Warehousing": ["Warehousing","Warehousing and support activities for transportation","Warehousing and support"],
    "Agriculture": ["Agriculture"],
    "Market Services": ["Market Services","Market services"],
    "Non Market Services": ["Non Market Services","Non-market services","Non Market services"],
}

all_sectors = set(df["sector"].dropna().astype(str).unique())
def _match_alias(alias_list, universe):
    for a in alias_list:
        if a in universe: return a
    lower_index = {s.lower(): s for s in universe}
    for a in alias_list:
        if a.lower() in lower_index: return lower_index[a.lower()]
    for a in alias_list:
        a_low = a.lower()
        for s in universe:
            if a_low in s.lower(): return s
    return None

target_map = {}
for canon, aliases in SECTOR_ALIASES.items():
    found = _match_alias(aliases, all_sectors)
    if found: target_map[canon] = found

if not target_map:
    raise RuntimeError("Could not map any sectors. Inspect df['sector'].unique().")

df = df[df["sector"].isin(target_map.values())].copy()
inv_map = {v:k for k,v in target_map.items()}
df["sector_canon"] = df["sector"].map(inv_map)

# -----------------------
# BLOCK 3 — Total loss per country×sector×hazard×year; take PEAKs; build HDW & SF
# -----------------------
agg = (df.groupby(["country","sector_canon","hazard","year"], dropna=False)
         .agg(total_loss=("value","sum")).reset_index())

base = agg[["country","sector_canon","year"]].drop_duplicates()

hazards = ["coastal_flood","river_flood","floods","storms","tropical_cyclone","winter_storm",
           "heatwave","drought","wildfire"]
for hz in hazards:
    sub = agg.loc[agg["hazard"].eq(hz), ["country","sector_canon","year","total_loss"]]
    base = base.merge(sub.rename(columns={"total_loss": f"tot_{hz}"}),
                      on=["country","sector_canon","year"], how="left")

def _rowmax(df_cols):
    if not df_cols: return np.full(len(base), np.nan)
    arr = np.stack([base[c].to_numpy() for c in df_cols], axis=1)
    return np.nanmax(arr, axis=1)

flood_cols  = [c for c in ["tot_floods","tot_river_flood","tot_coastal_flood"] if c in base.columns]
storm_cols  = [c for c in ["tot_storms","tot_tropical_cyclone","tot_winter_storm"] if c in base.columns]
base["tot_flood"]   = _rowmax(flood_cols)
base["tot_storms"]  = _rowmax(storm_cols)

peaks = (base.groupby(["country","sector_canon"], dropna=False)
            .agg(peak_heat=("tot_heatwave","max"),
                 peak_drought=("tot_drought","max"),
                 peak_wildfire=("tot_wildfire","max"),
                 peak_flood=("tot_flood","max"),
                 peak_storms=("tot_storms","max"))
            .reset_index())

peaks["HDW"] = peaks[["peak_heat","peak_drought","peak_wildfire"]].fillna(0).sum(axis=1)
peaks["SF"]  = peaks[["peak_storms","peak_flood"]].fillna(0).sum(axis=1)

# -----------------------
# BLOCK 4 — Country→Macro mapping (robust), then EU-15 & EU-27 means
# -----------------------
def _safe_norm(x):
    return _norm(x) if isinstance(x, str) else x

macro_map = pd.read_csv(MACRO_CSV, dtype=str)
macro_map["country"]      = macro_map["country"].map(_safe_norm).replace(alias).map(_safe_norm)
macro_map["macro_region"] = macro_map["macro_region"].map(_safe_norm)

# Add robust defaults for NGFS short-term buckets + basics if missing
need_explicit = {
    "USA": "USA",
    "Mexico": "Mexico",
    "Oceania": "Australia_NZ",
    "Rest of Europe": "Europe_Non_EU",
    "Rest of Energy Producing Countries": "Other",
    "Rest of World": "Other",
}
missing_rows = [k for k in need_explicit if not (macro_map["country"] == k).any()]
if missing_rows:
    macro_map = pd.concat(
        [macro_map,
         pd.DataFrame({
             "country": missing_rows,
             "macro_region": [need_explicit[k] for k in missing_rows]
         })],
        ignore_index=True
    )

macro_map = macro_map.drop_duplicates(subset=["country"], keep="first").reset_index(drop=True)

peaks["country"] = peaks["country"].map(_safe_norm)
peaks = peaks.merge(macro_map, on="country", how="left", validate="many_to_one")

# final catch-all: still-unmapped → 'Other' (and report)
if peaks["macro_region"].isna().any():
    still_missing = sorted(peaks.loc[peaks["macro_region"].isna(), "country"].unique())
    print("Auto-mapping remaining to 'Other':", still_missing)
    peaks.loc[peaks["macro_region"].isna(), "macro_region"] = "Other"

# Sum to macro for all macros
macro = (peaks.groupby(["macro_region","sector_canon"], dropna=False)
              .agg(HDW=("HDW","sum"), SF=("SF","sum")).reset_index())

# --- Replace 'Other' with MEAN across its member countries (consistency with EU means) ---
# Use `peaks`, which already includes `macro_region` from step 7.
other_mean = (peaks.loc[peaks["macro_region"] == "Other"]
              .groupby(["sector_canon"], dropna=False)
              .agg(HDW=("HDW","mean"), SF=("SF","mean"))
              .reset_index())

if not other_mean.empty:
    other_mean.insert(0, "macro_region", "Other")
    macro = pd.concat([macro[macro["macro_region"] != "Other"], other_mean], ignore_index=True)


# EU-means from country-level peaks (avoid inflation)
EU15 = ["Austria","Belgium","Denmark","Finland","France","Germany","Greece","Ireland","Italy",
        "Luxembourg","Netherlands","Portugal","Spain","Sweden","United Kingdom"]
EU27 = ["Austria","Belgium","Bulgaria","Croatia","Cyprus","Czechia","Denmark","Estonia","Finland",
        "France","Germany","Greece","Hungary","Ireland","Italy","Latvia","Lithuania","Luxembourg",
        "Malta","Netherlands","Poland","Portugal","Romania","Slovakia","Slovenia","Spain","Sweden"]

def _eu_mean(peaks_df, members, name):
    used = [c for c in members if c in set(peaks_df["country"])]
    if not used: return None
    out = (peaks_df[peaks_df["country"].isin(used)]
           .groupby(["sector_canon"], dropna=False)
           .agg(HDW=("HDW","mean"), SF=("SF","mean")).reset_index())
    out.insert(0, "macro_region", name)
    return out

eu15_rows = _eu_mean(peaks, EU15, "EU-15")
eu27_rows = _eu_mean(peaks, EU27, "EU-27")

macro_aug = macro.copy()
if "EU-27" in macro_aug["macro_region"].unique():
    macro_aug = macro_aug[macro_aug["macro_region"] != "EU-27"]
for extra in [eu15_rows, eu27_rows]:
    if extra is not None:
        macro_aug = pd.concat([macro_aug, extra], ignore_index=True)
        
# DiRe
macro_dire = macro_aug.copy()

# -----------------------
# BLOCK 5 — Heatmaps (same style & grid as DAPS)
# -----------------------
def _matrix(df, value_col, region_order, sector_order):
    p = df.pivot_table(index="macro_region", columns="sector_canon", values=value_col, aggfunc="sum")
    p = p.reindex(index=[r for r in region_order if r in p.index],
                  columns=[s for s in sector_order if s in p.columns])
    return p

region_order = [
    "USA","EU-15","EU-27","Europe_Non_EU","Japan","China","India","South Korea",
    "Canada","Australia_NZ","Mexico","Russia","Brazil",
    "South Africa","Argentina","Indonesia","Other"
]
sector_order = [
    "Power Supply","Ferrous metals","Non-metallic minerals","Construction",
    "Land transport","Water transport","Warehousing","Agriculture",
    "Market Services","Non Market Services",
]

M_HDW = _matrix(macro_aug, "HDW", region_order, sector_order)
M_SF  = _matrix(macro_aug, "SF",  region_order, sector_order)

_ensure_dir(OUTROOT / "data")
write_heatmap_tables(OUTROOT, "HDW_regionsxsectors_peak__DIRE", M_HDW)
write_heatmap_tables(OUTROOT, "SF_regionsxsectors_peak__DIRE",  M_SF)


def _p95(M):
    z = M.to_numpy(dtype=float)
    z = z[np.isfinite(z)]
    return float(np.percentile(z,95)) if z.size else 1.0

def _draw(ax, M, rows, cols, title, unit="percent"):
    if M.size == 0:
        ax.axis("off"); ax.set_title(f"{title}\n(no data)"); return
    vmin, vmax = 0.0, _p95(M)
    im = ax.imshow(M, aspect="auto", cmap="RdYlGn_r", vmin=vmin, vmax=vmax)
    ax.set_xticks(range(len(cols))); ax.set_xticklabels(cols, rotation=45, ha="right")
    ax.set_yticks(range(len(rows))); ax.set_yticklabels(rows)
    # “cadrillage”
    nrows, ncols = M.shape
    ax.set_xticks(np.arange(-0.5, ncols, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, nrows, 1), minor=True)
    ax.grid(which="minor", color="black", linewidth=0.4, linestyle="-")
    ax.tick_params(which="minor", bottom=False, left=False)
    c = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04,
                     format=FuncFormatter(lambda v, _pos: f"{v:.1f}"))
    c.ax.set_ylabel(f"Exposure ({unit})", rotation=270, va="bottom")
    ax.set_title(title, pad=10)

# HDW
fig1 = plt.figure(figsize=(12, max(5.0, 0.36*len(M_HDW.index)+2.2)))
ax1 = fig1.add_subplot(111)
_draw(ax1, M_HDW, list(M_HDW.index), list(M_HDW.columns),
      "Short-term physical risk (PEAK) — DIRE / HDW (heatwave + drought + wildfire)")
fig1.tight_layout()
png_hdw = OUTPNG / "total_loss__HDW__regionsxsectors__peak__DIRE.png"
fig1.savefig(png_hdw, dpi=240); plt.close(fig1)

# SF
fig2 = plt.figure(figsize=(12, max(5.0, 0.36*len(M_SF.index)+2.2)))
ax2 = fig2.add_subplot(111)
_draw(ax2, M_SF, list(M_SF.index), list(M_SF.columns),
      "Short-term physical risk (PEAK) — DIRE / SF (storms + flood)")
fig2.tight_layout()
png_sf = OUTPNG / "total_loss__SF__regionsxsectors__peak__DIRE.png"
fig2.savefig(png_sf, dpi=240); plt.close(fig2)

# Combined PDF
pdf = OUTPDF / "total_loss__HDW_and_SF__regionsxsectors__peak__DIRE.pdf"
with PdfPages(pdf) as pp:
    for png in [png_hdw, png_sf]:
        img = plt.imread(png)
        fig = plt.figure(figsize=(12,9)); plt.imshow(img); plt.axis('off')
        pp.savefig(fig); plt.close(fig)

print("[DIRE] PNG saved:", png_hdw)
print("[DIRE] PNG saved:", png_sf)
print("[DIRE] PDF saved:", pdf)


[DIRE] PNG saved: /Users/noenotter/Documents/Summer_intern/outputs/physical_shortterm_DIRE/png/total_loss__HDW__regionsxsectors__peak__DIRE.png
[DIRE] PNG saved: /Users/noenotter/Documents/Summer_intern/outputs/physical_shortterm_DIRE/png/total_loss__SF__regionsxsectors__peak__DIRE.png
[DIRE] PDF saved: /Users/noenotter/Documents/Summer_intern/outputs/physical_shortterm_DIRE/pdf/total_loss__HDW_and_SF__regionsxsectors__peak__DIRE.pdf


In [8]:
from pathlib import Path
import pandas as pd

def _ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)
    return p

def _to_long_from_matrix(mat_df: pd.DataFrame, index_name="Region", col_name="Sector", value_name="Value"):
    idx = mat_df.index.name or index_name
    return (mat_df.reset_index()
            .rename(columns={idx: index_name})
            .melt(id_vars=index_name, var_name=col_name, value_name=value_name))

def write_heatmap_tables(base_outdir: Path, tag: str, mat_df: pd.DataFrame):
    data_dir = _ensure_dir(base_outdir / "data")
    (data_dir / f"{tag}_heatmap_data.csv").write_text(mat_df.to_csv())
    _to_long_from_matrix(mat_df).to_csv(data_dir / f"{tag}_heatmap_data_long.csv", index=False)

import numpy as np, matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def _save_png_pdf_from_matrix(mat_df, png_path, pdf_path, title, vmin, vmax):
    rows, cols = list(mat_df.index), list(mat_df.columns)
    Mclip = np.clip(mat_df.to_numpy(float), vmin, vmax)

    # PNG
    fig, ax = plt.subplots(figsize=(1.2*len(cols), 0.48*len(rows)))
    im = ax.imshow(Mclip, aspect="auto", cmap="RdYlGn_r", vmin=vmin, vmax=vmax)
    ax.set_title(title, pad=10)
    ax.set_xticks(range(len(cols))); ax.set_xticklabels(cols, rotation=45, ha="right")
    ax.set_yticks(range(len(rows))); ax.set_yticklabels(rows)
    ax.set_xticks(np.arange(-.5, len(cols), 1), minor=True)
    ax.set_yticks(np.arange(-.5, len(rows), 1), minor=True)
    ax.grid(which="minor", color="black", linewidth=0.5, alpha=0.25)
    ax.tick_params(which="minor", bottom=False, left=False)
    fig.colorbar(im, ax=ax).set_label("Average physical loss (HDW & SF)")
    fig.tight_layout()
    fig.savefig(png_path, dpi=300, bbox_inches="tight")
    plt.close(fig)

    # one-page PDF
    with PdfPages(pdf_path) as pdf:
        fig2, ax2 = plt.subplots(figsize=(1.2*len(cols), 0.48*len(rows)))
        im2 = ax2.imshow(Mclip, aspect="auto", cmap="RdYlGn_r", vmin=vmin, vmax=vmax)
        ax2.set_title(title, pad=10)
        ax2.set_xticks(range(len(cols))); ax2.set_xticklabels(cols, rotation=45, ha="right")
        ax2.set_yticks(range(len(rows))); ax2.set_yticklabels(rows)
        ax2.set_xticks(np.arange(-.5, len(cols), 1), minor=True)
        ax2.set_yticks(np.arange(-.5, len(rows), 1), minor=True)
        ax2.grid(which="minor", color="black", linewidth=0.5, alpha=0.25)
        ax2.tick_params(which="minor", bottom=False, left=False)
        fig2.colorbar(im2, ax=ax2).set_label("Average physical loss (HDW & SF)")
        fig2.tight_layout()
        pdf.savefig(fig2, bbox_inches="tight")
        plt.close(fig2)
def _set_outdirs(model_tag: str):
    root = Path.home() / "Documents" / "Summer_intern" / "outputs" / f"physical_shortterm_{model_tag}"
    out_png = root / "png"; out_pdf = root / "pdf"
    out_png.mkdir(parents=True, exist_ok=True); out_pdf.mkdir(parents=True, exist_ok=True)
    return out_png, out_pdf

def _ensure_cols(df):
    needed = ["macro_region", "sector_canon"]
    for c in needed:
        if c not in df.columns:
            raise RuntimeError(f"Missing column '{c}' in DataFrame.")
    hdw = next((c for c in ["HDW","hdw","HDW_peak","HDW_sum"] if c in df.columns), None)
    sf  = next((c for c in ["SF","sf","SF_peak","SF_sum"] if c in df.columns), None)
    if hdw is None or sf is None:
        raise RuntimeError("Can't find HDW/SF columns.")
    return hdw, sf

def _infer_orders(df, region_order=None, sector_order=None):
    if region_order is None:
        region_order = sorted(df["macro_region"].dropna().unique().tolist())
    if sector_order is None:
        sector_order = sorted(df["sector_canon"].dropna().unique().tolist())
    return region_order, sector_order

def _clip_p95(arr):
    finite = arr[np.isfinite(arr)]
    if finite.size == 0: return arr, 0.0, 1.0
    p95 = np.percentile(finite, 95)
    vmax = float(p95 if p95 > 0 else np.nanmax(finite))
    if not np.isfinite(vmax) or vmax <= 0: vmax = 1.0
    return np.clip(arr, 0.0, vmax), 0.0, vmax

def _avg_heatmap(df_macro, model_tag: str, title: str, region_order=None, sector_order=None):
    """
    Build the average (HDW + SF)/2 Region×Sector matrix, save PNG/PDF, and write CSV wide+long.
    """
    OUTPNG, OUTPDF = _set_outdirs(model_tag)

    # columns & orders
    hdw_col, sf_col = _ensure_cols(df_macro)
    region_order, sector_order = _infer_orders(df_macro, region_order, sector_order)

    # average storyline
    df_avg = df_macro.copy()
    df_avg["ALL"] = (df_avg[hdw_col].astype(float) + df_avg[sf_col].astype(float)) / 2.0

    # drop non-present labels to avoid white rows/cols
    present_regions = set(df_avg["macro_region"].unique())
    present_sectors = set(df_avg["sector_canon"].unique())
    rows = [r for r in (region_order or []) if r in present_regions] or sorted(present_regions)
    cols = [s for s in (sector_order or []) if s in present_sectors] or sorted(present_sectors)
    miss_r = [r for r in (region_order or []) if r not in present_regions]
    miss_s = [s for s in (sector_order or []) if s not in present_sectors]
    if miss_r: print(f"[WARN:{model_tag}] Dropping missing regions from plot: {miss_r}")
    if miss_s: print(f"[WARN:{model_tag}] Dropping missing sectors from plot: {miss_s}")

    # pivot to Region×Sector
    mat = (df_avg.pivot(index="macro_region", columns="sector_canon", values="ALL")
                  .reindex(index=rows, columns=cols))
    if mat.isna().all().all():
        raise RuntimeError(f"[{model_tag}] Averaged matrix is all NaN after reindex.")

    # NEW: write CSVs for Streamlit
    _ensure_dir(OUTPNG.parent / "data")
    write_heatmap_tables(OUTPNG.parent, f"{model_tag}_Average_HDW_SF", mat)

    # common scale (p95), then render
    Mclip, vmin, vmax = _clip_p95(mat.to_numpy(float))
    png_path = OUTPNG / f"{model_tag}_Average_HDW_SF.png"
    pdf_path = OUTPDF / f"{model_tag}_Average_HDW_SF.pdf"
    _save_png_pdf_from_matrix(mat, png_path, pdf_path, title=title, vmin=vmin, vmax=vmax)
    print(f"[WRITE:{model_tag}] PNG -> {png_path}")
    print(f"[WRITE:{model_tag}] PDF -> {pdf_path}")


In [9]:
# === FINAL BLOCK: One averaged heatmap per model (DAPS & DiRe) ===
# Paste after: (i) you built DAPS macro table (macro_block4) and DiRe macro table (macro_aug),
# and (ii) any region_order / sector_order you may have defined.

import os, numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from pathlib import Path

# ---------- Resolve macro DataFrames ----------
# DAPS macro with EU-15/EU-27 means (from your Block 4A)
try:
    macro_daps = macro_block4.copy()
except NameError:
    macro_daps = None

# DiRe macro table (from your DiRe pipeline)
try:
    macro_dire = macro_aug.copy()
except NameError:
    macro_dire = None

# ---------- Optional: use your fixed orders if defined, else infer later ----------
try:
    _region_order = list(region_order)
except NameError:
    _region_order = None

try:
    _sector_order = list(sector_order)
except NameError:
    _sector_order = None

# ---------- Helpers ----------
def _set_outdirs(model_tag: str):
    root = Path.home() / "Documents" / "Summer_intern" / "outputs" / f"physical_shortterm_{model_tag}"
    out_png = root / "png"; out_pdf = root / "pdf"
    out_png.mkdir(parents=True, exist_ok=True); out_pdf.mkdir(parents=True, exist_ok=True)
    return out_png, out_pdf

def _ensure_cols(df):
    needed = ["macro_region", "sector_canon"]
    for c in needed:
        if c not in df.columns:
            raise RuntimeError(f"Missing column '{c}' in DataFrame. Has: {list(df.columns)[:25]} ...")
    # Accept common alternatives for the storylines
    hdw_candidates = ["HDW","hdw","HDW_peak","HDW_sum"]
    sf_candidates  = ["SF","sf","SF_peak","SF_sum"]
    hdw = next((c for c in hdw_candidates if c in df.columns), None)
    sf  = next((c for c in sf_candidates  if c in df.columns), None)
    if hdw is None or sf is None:
        raise RuntimeError(
            "Can't find HDW/SF columns.\n"
            f"Tried HDW in {hdw_candidates}, SF in {sf_candidates}.\n"
            f"Available: {list(df.columns)[:25]} ..."
        )
    return hdw, sf

def _infer_orders(df, region_order=None, sector_order=None):
    if region_order is None:
        region_order = sorted(df["macro_region"].dropna().unique().tolist())
    if sector_order is None:
        sector_order = sorted(df["sector_canon"].dropna().unique().tolist())
    return region_order, sector_order

def _clip_p95(arr):
    finite = arr[np.isfinite(arr)]
    if finite.size == 0:
        return arr, 0.0, 1.0
    p95 = np.percentile(finite, 95)
    vmax = float(p95 if p95 > 0 else np.nanmax(finite))
    if not np.isfinite(vmax) or vmax <= 0:
        vmax = 1.0
    return np.clip(arr, 0.0, vmax), 0.0, vmax


# ---------- RUN ----------
if macro_daps is not None and len(macro_daps):
    _avg_heatmap(
        df_macro=macro_daps,
        model_tag="DAPS",
        title="DAPS — Average of HDW & SF (peak 2023–2030)",
        region_order=_region_order,
        sector_order=_sector_order
    )
else:
    print("[SKIP] DAPS: macro_block4 not found or empty.")

if macro_dire is not None and len(macro_dire):
    _avg_heatmap(
        df_macro=macro_dire,
        model_tag="DIRE",
        title="DiRe — Average of HDW & SF (peak 2023–2030)",
        region_order=_region_order,
        sector_order=_sector_order
    )
else:
    print("[SKIP] DiRe: macro_aug not found or empty.")


[WRITE:DAPS] PNG -> /Users/noenotter/Documents/Summer_intern/outputs/physical_shortterm_DAPS/png/DAPS_Average_HDW_SF.png
[WRITE:DAPS] PDF -> /Users/noenotter/Documents/Summer_intern/outputs/physical_shortterm_DAPS/pdf/DAPS_Average_HDW_SF.pdf
[WRITE:DIRE] PNG -> /Users/noenotter/Documents/Summer_intern/outputs/physical_shortterm_DIRE/png/DIRE_Average_HDW_SF.png
[WRITE:DIRE] PDF -> /Users/noenotter/Documents/Summer_intern/outputs/physical_shortterm_DIRE/pdf/DIRE_Average_HDW_SF.pdf


In [10]:
# ====== NGFS short-term sector discovery + crosswalk to your 10 categories ======
import pandas as pd, numpy as np
from pathlib import Path

ROOT = Path.home() / "Documents" / "Summer_intern"
XLSX = ROOT / "NGFS_shortterm_FULL_timeseries.xlsx"

# Prefer these 4 families; fallback to whatever exists
PREF_SHEETS = ["capital_destruction","production_lost","productivity_loss","labour_productivity_loss"]

def load_ngfs_shortterm(xlsx_path: Path):
    xls = pd.ExcelFile(xlsx_path)
    sheets = [s for s in PREF_SHEETS if s in xls.sheet_names] or xls.sheet_names
    frames = []
    for sh in sheets:
        # Read only columns we need; be permissive about names
        df = pd.read_excel(xlsx_path, sheet_name=sh)
        df.columns = [str(c).strip().lower() for c in df.columns]
        # Normalize minimal set
        need = ["family","sector","model","scenario","region","unit","year","value","hazard"]
        for c in need:
            if c not in df.columns: df[c] = np.nan
        if df["family"].isna().all(): df["family"] = sh
        frames.append(df[need])
    full = pd.concat(frames, ignore_index=True)
    return full

full = load_ngfs_shortterm(XLSX)

# Keep only the physical-risk model ("direct_impacts") if present
mask_direct = full["model"].astype(str).str.lower().eq("direct_impacts")
if mask_direct.any():
    full = full[mask_direct].copy()

# 1) List sectors present
ngfs_sectors = sorted(full["sector"].dropna().astype(str).unique())
print(f"[INFO] NGFS sectors found ({len(ngfs_sectors)}):")
for s in ngfs_sectors:
    print(" -", s)

# 2) Crosswalk → your 10 categories (keyword rules, extend as needed)
CATS = {
    "Agriculture": ["agric", "biomass", "forestry", "fishing", "aquaculture"],
    "Mining": ["coal", "lignite", "crude", "petroleum", "natural gas", "oil & gas", "metal ore", "mining", "quarry"],
    "Consumer Goods": ["food", "beverage", "tobacco", "textile", "apparel", "garment", "leather", "wood", "paper", "printing", "publish"],
    "Heavy Industries": ["coke", "refined", "petroleum", "chemical", "pharm", "rubber", "plastic",
                         "non-metallic", "cement", "glass", "basic metal", "ferrous", "iron", "steel",
                         "non-ferrous", "fabricated metal"],
    "Equipment Goods": ["computer", "electronic", "optical", "electrical equip", "machinery",
                        "motor vehicle", "transport equipment", "furniture", "other manufacturing",
                        "repair", "installation", "battery", "pv", "solar", "wind", "ccs"],
    "Electricity production": ["electricity", "power", "gas supply", "steam", "air conditioning",
                               "hydrogen", "nuclear", "biomass", "hydro", "geothermal"],
    "Construction": ["construct", "civil engineering", "special"],
    "Transport": ["transport", "land", "road", "rail", "pipeline", "water", "maritime", "air", "aviation"],
    "Warehousing": ["warehous", "postal", "courier", "support activit"],
    "Services": ["accommodation", "food service", "financial", "insurance", "auxiliary",
                 "real estate", "information", "telecom", "publishing", "it ", " r&d", "research",
                 "legal", "accounting", "consult", "engineering", "scientific", "advertis",
                 "veterinary", "leasing", "employment", "travel agency", "security",
                 "building service", "admin", "membership", "repair of computers",
                 "personal service", "households as employers", "extraterritorial",
                 "education", "public administration", "health", "social work",
                 "waste", "water supply", "sewerage", "remediation", "gambling", "arts", "sports",
                 "wholesale", "retail"]
}

def to_category(label: str) -> str | None:
    lab = str(label).lower()
    # Exact helpful heuristics first
    if "electric" in lab and "supply" in lab: return "Electricity production"
    if "power" in lab: return "Electricity production"
    # Generic keyword scan
    for cat, keys in CATS.items():
        for k in keys:
            if k in lab:
                return cat
    return None

crosswalk = pd.DataFrame({"NGFS_sector_raw": ngfs_sectors})
crosswalk["Proposed_10cat"] = crosswalk["NGFS_sector_raw"].apply(to_category)

unmatched = crosswalk[crosswalk["Proposed_10cat"].isna()].copy()
if len(unmatched):
    print("\n[WARN] Unmatched sectors (add aliases):")
    for s in unmatched["NGFS_sector_raw"]:
        print(" -", s)
else:
    print("\n[OK] All NGFS sectors mapped to a 10-category.")

# 3) If you want an averaged-by-10cat view ready for heatmaps:
def collapse_to_10cat(df: pd.DataFrame, crosswalk_df: pd.DataFrame) -> pd.DataFrame:
    # Join category on sector
    cw = crosswalk_df.dropna(subset=["Proposed_10cat"]).copy()
    df2 = df.merge(cw, left_on="sector", right_on="NGFS_sector_raw", how="left")
    # Keep only rows we can place
    df2 = df2[~df2["Proposed_10cat"].isna()].copy()
    # Example aggregation: total_loss by country×10cat×hazard×year (like your pipeline upstream)
    # Here we just show you what categories are present per scenario/model
    present = (df2.groupby(["scenario","model","Proposed_10cat"], dropna=False)
                  .size().reset_index(name="rows"))
    return df2, present

collapsed_df, present_table = collapse_to_10cat(full, crosswalk)

print("\n[INFO] Rows per (scenario, model, 10cat) present in the workbook:")
print(present_table.sort_values(["scenario","model","Proposed_10cat"]).to_string(index=False))

# If you want to export the crosswalk for manual touch-up:
crosswalk_path = ROOT / "ngfs_sector_to_10cat_crosswalk.csv"
crosswalk.to_csv(crosswalk_path, index=False)
print(f"\n[WRITE] Crosswalk CSV -> {crosswalk_path}")


[INFO] NGFS sectors found (50):
 - Advanced Electric Appliances
 - Advanced Heating and Cooking Appliances
 - Agriculture
 - Air transport
 - Basic pharmaceutical products
 - Batteries
 - Biofuels
 - Biomass
 - Biomass Solid
 - CCS coal
 - CO2 Capture
 - CSS Bio
 - CSS Gas
 - Chemical Products
 - Clean Gas
 - Coal
 - Coal fired
 - Computer, electronic and optical products
 - Construction
 - Consumer Goods Industries
 - Crude Oil
 - EV Transport Equipment
 - Equipment for CCS power technology
 - Equipment for PV panels
 - Equipment for wind power technology
 - Fabricated Metal products
 - Ferrous metals
 - Gas
 - Gas Fired
 - Geothermal
 - Hydro electric
 - Hydrogen
 - Land transport
 - Market Services
 - Non Market Services
 - Non-ferrous metals
 - Non-metallic minerals
 - Nuclear
 - Oil
 - Oil Fired
 - Other Equipment Goods
 - PV
 - Paper products, publishing
 - Power Supply
 - R&D
 - Rubber and plastic products
 - Transport equipment (excluding EV)
 - Warehousing
 - Water transport
 

In [11]:
# =============================
# BLOCK X — 10-category sector heatmaps (mean across subsectors), DAPS & DiRe
# Paste AFTER your existing blocks (after the DiRe pipeline + average block).
# =============================

import pandas as pd, numpy as np, unicodedata
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import FuncFormatter

# ---- 10 categories in your desired order (Overleaf Table A.2) ----
TEN_ORDER = [
    "Agriculture","Mining","Consumer Goods","Heavy Industries","Equipment Goods",
    "Electricity production","Construction","Transport","Warehousing","Services",
]

# ---- Explicit NGFS subsector -> 10-category mapping (covers your labels) ----
NGFS_TO_10CAT = {
    # Agriculture
    "Agriculture":"Agriculture","Biomass":"Agriculture","Biomass Solid":"Agriculture",
    # Mining / fuels upstream
    "Coal":"Mining","Crude Oil":"Mining","Oil":"Mining","Gas":"Mining","Clean Gas":"Mining",
    # Consumer Goods
    "Consumer Goods Industries":"Consumer Goods","Paper products, publishing":"Consumer Goods",
    # Heavy Industries
    "Chemical Products":"Heavy Industries","Basic pharmaceutical products":"Heavy Industries",
    "Rubber and plastic products":"Heavy Industries","Non-metallic minerals":"Heavy Industries",
    "Ferrous metals":"Heavy Industries","Non-ferrous metals":"Heavy Industries",
    "Fabricated Metal products":"Heavy Industries",
    # Equipment Goods
    "Computer, electronic and optical products":"Equipment Goods",
    "Advanced Electric Appliances":"Equipment Goods",
    "Advanced Heating and Cooking Appliances":"Equipment Goods",
    "Transport equipment (excluding EV)":"Equipment Goods",
    "EV Transport Equipment":"Equipment Goods","Other Equipment Goods":"Equipment Goods",
    "Batteries":"Equipment Goods","Equipment for PV panels":"Equipment Goods",
    "Equipment for wind power technology":"Equipment Goods",
    "Equipment for CCS power technology":"Equipment Goods",
    # Electricity production
    "Power Supply":"Electricity production","Hydro electric":"Electricity production",
    "Wind":"Electricity production","PV":"Electricity production","Geothermal":"Electricity production",
    "Nuclear":"Electricity production","Coal fired":"Electricity production",
    "Oil Fired":"Electricity production","Gas Fired":"Electricity production",
    "Biofuels":"Electricity production","Hydrogen":"Electricity production",
    "CO2 Capture":"Electricity production","CCS coal":"Electricity production",
    "CSS Gas":"Electricity production","CSS Bio":"Electricity production",
    # Construction
    "Construction":"Construction",
    # Transport
    "Land transport":"Transport","Water transport":"Transport","Air transport":"Transport",
    # Warehousing
    "Warehousing":"Warehousing",
    # Services
    "Market Services":"Services","Non Market Services":"Services","R&D":"Services",
}

# ---- Small helpers (re-use existing ones if already defined globally) ----
def _normstrong(s: str) -> str:
    s = unicodedata.normalize("NFKC", str(s))
    s = s.replace("\u00A0"," ").replace("\u200B","").replace("\u200C","").replace("\u200D","")
    return " ".join(s.split()).strip()

_alias = {"United States":"USA","U.S.A.":"USA","US":"USA","United States of America":"USA",
          "Korea, Republic of":"South Korea","Republic of Korea":"South Korea",
          "Russian Federation":"Russia","Viet Nam":"Vietnam"}

def _macro_aggregate_with_eu_means_10cat(peaks_df: pd.DataFrame, MACRO_CSV: Path) -> pd.DataFrame:
    mm = pd.read_csv(MACRO_CSV, dtype=str)
    mm["country"] = mm["country"].astype(str).map(_normstrong).replace(_alias).map(_normstrong)
    mm["macro_region"] = mm["macro_region"].astype(str).map(_normstrong)

    essentials = {
        "USA":"USA","Mexico":"Mexico",
        "Australia":"Australia_NZ","New Zealand":"Australia_NZ",
        "Switzerland":"Europe_Non_EU","Norway":"Europe_Non_EU","Iceland":"Europe_Non_EU","Turkey":"Europe_Non_EU",
        "Oceania":"Australia_NZ","Rest of Europe":"Europe_Non_EU",
        "Rest of Energy Producing Countries":"Other","Rest of World":"Other",
    }
    for ctry, macro in essentials.items():
        if not (mm["country"] == ctry).any():
            mm.loc[len(mm)] = {"country": ctry, "macro_region": macro}

    dfm = peaks_df.copy()
    dfm["country"] = dfm["country"].astype(str).map(_normstrong).replace(_alias).map(_normstrong)
    dfm = dfm.merge(mm, on="country", how="left", validate="many_to_one")
    if dfm["macro_region"].isna().any():
        missing = sorted(dfm.loc[dfm["macro_region"].isna(),"country"].unique())
        print("[NOTE] Auto-assigning to 'Other':", missing)
        dfm.loc[dfm["macro_region"].isna(), "macro_region"] = "Other"

    # Macro aggregation
    macro = (dfm.groupby(["macro_region","sector_10cat"], dropna=False)
               .agg(HDW=("HDW","sum"), SF=("SF","sum")).reset_index())

    # Force 'Other' = mean of its members
    other_mean = (dfm.loc[dfm["macro_region"]=="Other"]
                    .groupby(["sector_10cat"], dropna=False)
                    .agg(HDW=("HDW","mean"), SF=("SF","mean")).reset_index())
    if not other_mean.empty:
        other_mean.insert(0, "macro_region", "Other")
        macro = pd.concat([macro[macro["macro_region"]!="Other"], other_mean], ignore_index=True)

    # EU15 / EU27 means
    EU15 = ["Austria","Belgium","Denmark","Finland","France","Germany","Greece","Ireland","Italy",
            "Luxembourg","Netherlands","Portugal","Spain","Sweden","United Kingdom"]
    EU27 = ["Austria","Belgium","Bulgaria","Croatia","Cyprus","Czechia","Denmark","Estonia","Finland",
            "France","Germany","Greece","Hungary","Ireland","Italy","Latvia","Lithuania","Luxembourg",
            "Malta","Netherlands","Poland","Portugal","Romania","Slovakia","Slovenia","Spain","Sweden"]

    def _eu_mean(peaks_country_df, members, name):
        used = [c for c in members if c in set(peaks_country_df["country"])]
        if not used: return None
        sub = peaks_country_df[peaks_country_df["country"].isin(used)]
        out = (sub.groupby(["sector_10cat"], dropna=False)
                  .agg(HDW=("HDW","mean"), SF=("SF","mean")).reset_index())
        out.insert(0, "macro_region", name)
        return out

    eu15 = _eu_mean(dfm, EU15, "EU-15"); eu27 = _eu_mean(dfm, EU27, "EU-27")
    macro_aug = macro.copy()
    if "EU-27" in set(macro_aug["macro_region"]):
        macro_aug = macro_aug[macro_aug["macro_region"] != "EU-27"]
    for extra in (eu15, eu27):
        if extra is not None:
            macro_aug = pd.concat([macro_aug, extra], ignore_index=True)

    macro_aug = (macro_aug
                 .groupby(["macro_region","sector_10cat"], as_index=False)
                 .agg(HDW=("HDW","mean"), SF=("SF","mean")))
    return macro_aug

def _collapse_to_10cat_and_peaks(df_src: pd.DataFrame) -> pd.DataFrame:
    """Equal-weight collapse of NGFS subsectors into 10 categories, then PEAKs & storylines."""
    d = df_src.copy()
    d.columns = [str(c).strip().lower() for c in d.columns]
    for c in ["country","sector","hazard","year","value","scenario","model","region"]:
        if c not in d.columns: d[c] = np.nan

    d["sector"] = d["sector"].astype(str)
    d["sector_10cat"] = d["sector"].map(NGFS_TO_10CAT)
    if d["sector_10cat"].isna().any():
        missing_labels = sorted(d.loc[d["sector_10cat"].isna(),"sector"].unique().tolist())
        print("[WARN] Unmapped sector labels dropped:", missing_labels)
        d = d[~d["sector_10cat"].isna()].copy()

    g = (d.groupby(["country","sector_10cat","hazard","year"], dropna=False)["value"]
           .mean().reset_index(name="total_loss"))

    base = g[["country","sector_10cat","year"]].drop_duplicates()
    haz_list = ["coastal_flood","river_flood","floods","storms","tropical_cyclone","winter_storm",
                "heatwave","drought","wildfire"]
    for hz in haz_list:
        sub = g[g["hazard"].eq(hz)][["country","sector_10cat","year","total_loss"]]
        base = base.merge(sub.rename(columns={"total_loss": f"tot_{hz}"}),
                          on=["country","sector_10cat","year"], how="left")

    def _rowmax(cols):
        arrs = [base[c].to_numpy() for c in cols if c in base]
        if not arrs: return np.full(len(base), np.nan)
        A = np.vstack(arrs).T
        return np.nanmax(A, axis=1)

    base["tot_flood"]   = _rowmax(["tot_floods","tot_river_flood","tot_coastal_flood"])
    base["tot_stormsC"] = _rowmax(["tot_storms","tot_tropical_cyclone","tot_winter_storm"])

    peaks = (base.groupby(["country","sector_10cat"], dropna=False)
                  .agg(peak_storms=("tot_stormsC","max"),
                       peak_flood=("tot_flood","max"),
                       peak_heat=("tot_heatwave","max"),
                       peak_drought=("tot_drought","max"),
                       peak_wildfire=("tot_wildfire","max")).reset_index())

    for col in ["peak_storms","peak_flood","peak_heat","peak_drought","peak_wildfire"]:
        if col not in peaks:
            peaks[col] = np.nan
    
    peaks["HDW"] = peaks[["peak_heat","peak_drought","peak_wildfire"]].fillna(0).sum(axis=1)
    peaks["SF"]  = peaks[["peak_storms","peak_flood"]].fillna(0).sum(axis=1)

    # 🚫 remove Taiwan here
    peaks["country"] = peaks["country"].astype(str).str.strip()
    peaks = peaks[~peaks["country"].str.fullmatch("Taiwan", case=False)].copy()

    return peaks

def _plot_heatmap(Mdf: pd.DataFrame, title, out_png, out_pdf):
    vals = Mdf.to_numpy(dtype=float).ravel()
    vals = vals[np.isfinite(vals)]
    vmax = float(np.percentile(vals,95)) if vals.size else 1.0
    vmin = 0.0

    rows, cols = list(Mdf.index), list(Mdf.columns)
    fig, ax = plt.subplots(figsize=(1.2*len(cols), 0.48*len(rows)))
    im = ax.imshow(Mdf.to_numpy(float), aspect="auto", cmap="RdYlGn_r", vmin=vmin, vmax=vmax)
    ax.set_xticks(range(len(cols))); ax.set_xticklabels(cols, rotation=45, ha="right")
    ax.set_yticks(range(len(rows))); ax.set_yticklabels(rows)
    ax.set_xticks(np.arange(-0.5, len(cols), 1), minor=True)
    ax.set_yticks(np.arange(-0.5, len(rows), 1), minor=True)
    ax.grid(which="minor", color="black", linewidth=0.4)
    ax.tick_params(which="minor", bottom=False, left=False)
    c = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04,
                     format=FuncFormatter(lambda v,_: f"{v:.1f}"))
    c.ax.set_ylabel("Exposure (units of input)", rotation=270, va="bottom")
    ax.set_title(title, pad=10)
    fig.tight_layout(); fig.savefig(out_png, dpi=300, bbox_inches="tight"); plt.close(fig)

    with PdfPages(out_pdf) as pdf:
        fig2, ax2 = plt.subplots(figsize=(1.2*len(cols), 0.48*len(rows)))
        im2 = ax2.imshow(Mdf.to_numpy(float), aspect="auto", cmap="RdYlGn_r", vmin=vmin, vmax=vmax)
        ax2.set_xticks(range(len(cols))); ax2.set_xticklabels(cols, rotation=45, ha="right")
        ax2.set_yticks(range(len(rows))); ax2.set_yticklabels(rows)
        ax2.set_xticks(np.arange(-0.5, len(cols), 1), minor=True)
        ax2.set_yticks(np.arange(-0.5, len(rows), 1), minor=True)
        ax2.grid(which="minor", color="black", linewidth=0.4)
        ax2.tick_params(which="minor", bottom=False, left=False)
        c2 = plt.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04,
                          format=FuncFormatter(lambda v,_: f"{v:.1f}"))
        c2.ax.set_ylabel("Exposure (units of input)", rotation=270, va="bottom")
        ax2.set_title(title, pad=10)
        fig2.tight_layout(); pdf.savefig(fig2, bbox_inches="tight"); plt.close(fig2)

def _run_model_10cat(df_src, model_tag: str, out_root_tag: str,
                     region_order=None, sector10_order=None):
    if df_src is None or df_src.empty:
        print(f"[SKIP] {model_tag}: no source data found.")
        return

    # 1) collapse to 10-cat & compute HDW/SF peaks per country
    peaks_10 = _collapse_to_10cat_and_peaks(df_src)

    # 2) aggregate to macro with EU means
    macro10 = _macro_aggregate_with_eu_means_10cat(peaks_10, MACRO_CSV)

    # 3) determine orders
    if region_order is None:
        region_order = sorted(macro10["macro_region"].unique().tolist())
    if sector10_order is None:
        sector10_order = [c for c in TEN_ORDER if c in set(macro10["sector_10cat"])]

    # 4) matrices as DataFrames
    def _mat_df(val):
        return (macro10.pivot(index="macro_region", columns="sector_10cat", values=val)
                       .reindex(index=region_order, columns=sector10_order))

    M10_HDW_df = _mat_df("HDW")
    M10_SF_df  = _mat_df("SF")

    # 5) out dirs
    ROOT = Path.home() / "Documents" / "Summer_intern" / "outputs" / f"physical_shortterm_{out_root_tag}"
    OUTPNG = ROOT / "png_10cats"; OUTPDF = ROOT / "pdf_10cats"; OUTDATA = ROOT / "data_10cats"
    OUTPNG.mkdir(parents=True, exist_ok=True); OUTPDF.mkdir(parents=True, exist_ok=True); OUTDATA.mkdir(parents=True, exist_ok=True)

    # 6) plots
    _plot_heatmap(M10_HDW_df, f"{model_tag} — 10 categories — HDW (peak 2023–2030)",
                  OUTPNG / f"{model_tag}__HDW__10cats__peak.png",
                  OUTPDF / f"{model_tag}__HDW__10cats__peak.pdf")
    _plot_heatmap(M10_SF_df, f"{model_tag} — 10 categories — SF (peak 2023–2030)",
                  OUTPNG / f"{model_tag}__SF__10cats__peak.png",
                  OUTPDF / f"{model_tag}__SF__10cats__peak.pdf")

    # 7) write CSVs
    def _to_long(df: pd.DataFrame):
        idx = df.index.name or "Region"
        return (df.reset_index()
                .rename(columns={idx:"Region"})
                .melt(id_vars="Region", var_name="Sector", value_name="Value"))

    (OUTDATA / "HDW_10cats_peak_heatmap_data.csv").write_text(M10_HDW_df.to_csv())
    _to_long(M10_HDW_df).to_csv(OUTDATA / "HDW_10cats_peak_heatmap_data_long.csv", index=False)
    (OUTDATA / "SF_10cats_peak_heatmap_data.csv").write_text(M10_SF_df.to_csv())
    _to_long(M10_SF_df).to_csv(OUTDATA / "SF_10cats_peak_heatmap_data_long.csv", index=False)

    # 8) Average (HDW & SF)
    M10_AVG_df = (M10_HDW_df.add(M10_SF_df, fill_value=0.0)) / 2.0
    _plot_heatmap(M10_AVG_df, f"{model_tag} — 10 categories — Average (HDW & SF)",
                  OUTPNG / f"{model_tag}__AVG_HDW_SF__10cats.png",
                  OUTPDF / f"{model_tag}__AVG_HDW_SF__10cats.pdf")
    (OUTDATA / "AVG_HDW_SF_10cats_heatmap_data.csv").write_text(M10_AVG_df.to_csv())
    _to_long(M10_AVG_df).to_csv(OUTDATA / "AVG_HDW_SF_10cats_heatmap_data_long.csv", index=False)

    print(f"[WRITE:{model_tag}] 10-category heatmaps saved to: {OUTPNG} and {OUTPDF}")
    print(f"[WRITE:{model_tag}] 10-category data saved to: {OUTDATA}")

# ---- Source datasets ----
try:
    df_src_daps = df_block1.copy()
except NameError:
    df_src_daps = None

try:
    _full0 = full0.copy()
except NameError:
    def _rebuild_full0(_IN_XLSX):
        FAMILY_SHEETS = ["capital_destruction","production_lost","productivity_loss","labour_productivity_loss"]
        parts = []
        for sh in FAMILY_SHEETS:
            df = pd.read_excel(_IN_XLSX, sheet_name=sh,
                               usecols=["family","hazard","sector","model","scenario","region","unit","year","value"])
            df.columns = [c.strip().lower() for c in df.columns]
            if "family" not in df.columns: 
                df["family"] = sh
            parts.append(df)
        f0 = pd.concat(parts, ignore_index=True)
        f0["year"]  = pd.to_numeric(f0["year"], errors="coerce").astype("Int64")
        f0["value"] = pd.to_numeric(f0["value"], errors="coerce")
        f0 = f0[f0["year"].between(2023, 2030)]
        f0 = f0[f0["model"].fillna("").str.lower().eq("direct_impacts")].copy()
        f0["country"] = f0["region"].astype(str).str.split(" - ").str[0].str.strip()
        return f0
    _full0 = _rebuild_full0(IN_XLSX)

df_src_dire = _full0[_full0["scenario"].astype(str).str.strip().str.lower().eq("dire")].copy()

# ---- Run for DAPS and DiRe ----
try:
    _CANON_REGIONS = _region_order  # reuse if already defined upstream
except NameError:
    _CANON_REGIONS = None

_run_model_10cat(df_src_daps, "DAPS", "DAPS", region_order=_CANON_REGIONS, sector10_order=TEN_ORDER)
_run_model_10cat(df_src_dire, "DiRe",  "DIRE", region_order=_CANON_REGIONS, sector10_order=TEN_ORDER)



[WRITE:DAPS] 10-category heatmaps saved to: /Users/noenotter/Documents/Summer_intern/outputs/physical_shortterm_DAPS/png_10cats and /Users/noenotter/Documents/Summer_intern/outputs/physical_shortterm_DAPS/pdf_10cats
[WRITE:DAPS] 10-category data saved to: /Users/noenotter/Documents/Summer_intern/outputs/physical_shortterm_DAPS/data_10cats
[WRITE:DiRe] 10-category heatmaps saved to: /Users/noenotter/Documents/Summer_intern/outputs/physical_shortterm_DIRE/png_10cats and /Users/noenotter/Documents/Summer_intern/outputs/physical_shortterm_DIRE/pdf_10cats
[WRITE:DiRe] 10-category data saved to: /Users/noenotter/Documents/Summer_intern/outputs/physical_shortterm_DIRE/data_10cats


In [12]:
# ===============================
# DIAGNOSTICS: "Other" breakdown, scales, and India-Construction checks
# Run this AFTER your existing blocks (the ones that define peaks_block3, macro_block4, macro_aug, etc.)
# ===============================
import pandas as pd, numpy as np
from pathlib import Path

# --- Helper: p95
def _p95_np(A):
    z = A[np.isfinite(A)]
    return float(np.percentile(z, 95)) if z.size else 1.0

# --- 0) Which countries ended up in "Other" (original 10-sector pipeline)?
try:
    # Recreate the country→macro mapping used earlier
    macro_map = pd.read_csv(MACRO_CSV, dtype=str)  # defined in your code
    # Normalize like Block 4 did
    def _norm(s: str) -> str:
        import unicodedata
        s = unicodedata.normalize("NFKC", str(s))
        s = s.replace("\u00A0"," ").replace("\u200B","").replace("\u200C","").replace("\u200D","")
        return " ".join(s.split()).strip()
    alias = {"United States":"USA","U.S.A.":"USA","US":"USA","United States of America":"USA",
             "Korea, Republic of":"South Korea","Republic of Korea":"South Korea",
             "Russian Federation":"Russia","Viet Nam":"Vietnam"}
    macro_map["country"] = macro_map["country"].astype(str).map(_norm).replace(alias).map(_norm)
    macro_map["macro_region"] = macro_map["macro_region"].astype(str).map(_norm)

    # peaks_block3 = country-level peaks from your original 10-sector pipeline
    peaks_countries = peaks_block3.copy()
    peaks_countries["country"] = peaks_countries["country"].astype(str).map(_norm).replace(alias).map(_norm)
    peaks_join = peaks_countries.merge(macro_map, on="country", how="left")

    other_cty = (peaks_join[peaks_join["macro_region"] == "Other"]
                 .groupby("country", as_index=False)[["HDW","SF"]].sum()
                 .sort_values(["HDW","SF"], ascending=False))
    print("\n[OTHER] Top contributing countries (original 10 sectors):")
    print(other_cty.head(10).to_string(index=False))

    # Optional: share within 'Other'
    total_other = other_cty[["HDW","SF"]].sum()
    if total_other.sum() > 0:
        tmp = other_cty.copy()
        tmp["HDW_share_%"] = 100 * tmp["HDW"] / (total_other["HDW"] if total_other["HDW"] else 1)
        tmp["SF_share_%"]  = 100 * tmp["SF"]  / (total_other["SF"]  if total_other["SF"]  else 1)
        print("\n[OTHER] Shares within 'Other' (%):")
        print(tmp.head(10)[["country","HDW_share_%","SF_share_%"]].to_string(index=False))
except Exception as e:
    print("[WARN] Could not compute 'Other' breakdown for original pipeline:", e)

# --- 1) Matrix scales (original 10-sector pipeline)
try:
    # M_HDW and M_SF already built in your Block 5 for both DAPS and DiRe
    def _matrix_stats(M, label):
        A = M.to_numpy(float) if hasattr(M, "to_numpy") else np.array(M, float)
        print(f"[SCALE] {label}: p95={_p95_np(A):.3f}, max={np.nanmax(A):.3f}")
    print("\n[SCALES] Original 10 sectors — DAPS:")
    _matrix_stats(M_HDW, "DAPS HDW")
    _matrix_stats(M_SF,  "DAPS SF")
except Exception as e:
    print("[WARN] Original 10-sector scale check skipped:", e)

# --- 2) India – Construction comparison (original pipeline)
try:
    india_constr = peaks_countries[(peaks_countries["country"]=="India") &
                                   (peaks_countries["sector_canon"]=="Construction")]
    print("\n[CHECK] India–Construction (original 10 sectors):")
    print(india_constr[["HDW","SF"]].describe(percentiles=[]))
except Exception as e:
    print("[WARN] India–Construction (original) check skipped:", e)

# === 10-category pipeline diagnostics (uses functions from your BLOCK X) ===
try:
    # Rebuild 10-category country peaks from the same source DAPS Block 1
    # df_block1 = your per-country chosen DAPS data
    from math import isfinite
    peaks_10_daps = _collapse_to_10cat_and_peaks(df_block1)
    peaks_10_dire = _collapse_to_10cat_and_peaks(full0[full0["scenario"].astype(str).str.lower().eq("dire")])

    # 'Other' breakdown for 10-categories (DAPS)
    mm = pd.read_csv(MACRO_CSV, dtype=str)
    mm["country"] = mm["country"].astype(str).map(_norm).replace(alias).map(_norm)
    mm["macro_region"] = mm["macro_region"].astype(str).map(_norm)
    pj = peaks_10_daps.copy()
    pj["country"] = pj["country"].astype(str).map(_norm).replace(alias).map(_norm)
    pj = pj.merge(mm, on="country", how="left")
    other10 = (pj[pj["macro_region"]=="Other"]
               .groupby("country", as_index=False)[["HDW","SF"]].sum()
               .sort_values(["HDW","SF"], ascending=False))
    print("\n[OTHER] Top contributing countries (10-category DAPS):")
    print(other10.head(10).to_string(index=False))
except Exception as e:
    print("[WARN] 10-category 'Other' breakdown skipped:", e)

# --- 3) Build 10-category macro matrices and scale stats (uses your helper)
try:
    macro10_daps = _macro_aggregate_with_eu_means(peaks_10_daps, MACRO_CSV)
    rows = [r for r in region_order if r in set(macro10_daps["macro_region"])]
    cols = [c for c in TEN_ORDER if c in set(macro10_daps["sector_10cat"])]
    M10_HDW = macro10_daps.pivot(index="macro_region", columns="sector_10cat", values="HDW").reindex(index=rows, columns=cols)
    M10_SF  = macro10_daps.pivot(index="macro_region", columns="sector_10cat", values="SF" ).reindex(index=rows, columns=cols)
    print("\n[SCALES] 10-category — DAPS:")
    _matrix_stats(M10_HDW, "DAPS HDW (10-cat)")
    _matrix_stats(M10_SF,  "DAPS SF  (10-cat)")

    # India–Construction check in 10-cat (same name)
    india_constr_10 = peaks_10_daps[(peaks_10_daps["country"]=="India") &
                                    (peaks_10_daps["sector_10cat"]=="Construction")]
    print("\n[CHECK] India–Construction (10 categories):")
    print(india_constr_10[["HDW","SF"]].describe(percentiles=[]))
except Exception as e:
    print("[WARN] 10-category scale/India checks skipped:", e)

# --- 4) Optional: choose how to compute "combined" (mean vs sum)
COMBINED_MODE = "mean"  # change to "sum" to align with Eric's suggestion
def combine_storylines(H, S, mode="mean"):
    A = np.array(H, float); B = np.array(S, float)
    if mode == "sum":  return A + B
    return 0.5*(A + B)

try:
    M10_AVG = combine_storylines(M10_HDW.to_numpy(float), M10_SF.to_numpy(float), mode=COMBINED_MODE)
    print(f"\n[COMBINED] Mode={COMBINED_MODE}: p95={_p95_np(M10_AVG):.3f}, max={np.nanmax(M10_AVG):.3f}")
except Exception as e:
    print("[WARN] Combined check skipped:", e)

# --- 5) (Optional) Recompute 'Other' as MEAN instead of SUM for fairness
TRY_OTHER_AS_MEAN = False
if TRY_OTHER_AS_MEAN:
    # Recompute macro10_daps but averaging within each macro (including Other)
    pj = peaks_10_daps.merge(mm, on="country", how="left")
    macro10_mean = (pj.groupby(["macro_region","sector_10cat"], dropna=False)
                      .agg(HDW=("HDW","mean"), SF=("SF","mean")).reset_index())
    M10m_HDW = macro10_mean.pivot("macro_region","sector_10cat","HDW").reindex(index=rows, columns=cols)
    print("\n[SCALES] 10-category — DAPS with macro means everywhere (incl. Other):")
    _matrix_stats(M10m_HDW, "DAPS HDW (10-cat, macro means)")




[OTHER] Top contributing countries (original 10 sectors):
     country       HDW       SF
Saudi Arabia 43.605519 4.574846
      Turkey 27.636146 4.279562

[OTHER] Shares within 'Other' (%):
     country  HDW_share_%  SF_share_%
Saudi Arabia    61.207889   51.667439
      Turkey    38.792111   48.332561

[SCALES] Original 10 sectors — DAPS:
[SCALE] DAPS HDW: p95=12.051, max=34.224
[SCALE] DAPS SF: p95=4.624, max=5.391

[CHECK] India–Construction (original 10 sectors):
             HDW   SF
count   1.000000  1.0
mean   27.221689  0.0
std          NaN  NaN
min    27.221689  0.0
50%    27.221689  0.0
max    27.221689  0.0

[OTHER] Top contributing countries (10-category DAPS):
     country       HDW       SF
Saudi Arabia 10.719337 1.724093
      Turkey  5.425557 1.612812
[WARN] 10-category scale/India checks skipped: name '_macro_aggregate_with_eu_means' is not defined
[WARN] Combined check skipped: name 'M10_HDW' is not defined
